## 1. Importing needed libraries

In [1]:
import librosa
import librosa.display
import keras
import random
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

Using TensorFlow backend.


<hr>

#### GPU activation for Windows Machine with Nvidia only

In [2]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

ImportError: Traceback (most recent call last):
  File "/Users/amir/anaconda3/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 41, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/Users/amir/anaconda3/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/Users/amir/anaconda3/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/Users/amir/anaconda3/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/Users/amir/anaconda3/lib/python3.6/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: dlopen(/Users/amir/anaconda3/lib/python3.6/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so, 10): Library no

In [3]:
# Check that there is an output
from keras import backend as k
k.tensorflow_backend._get_available_gpus()

ModuleNotFoundError: No module named 'keras'

<hr>

## 2. Data Preparation

In [2]:
data = pd.read_csv('data/metadata.csv')
data.head(10)

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.000000,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.500000,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.500000,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.000000,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.500000,72.500000,1,5,2,children_playing
5,100263-2-0-143.wav,100263,71.500000,75.500000,1,5,2,children_playing
6,100263-2-0-161.wav,100263,80.500000,84.500000,1,5,2,children_playing
7,100263-2-0-3.wav,100263,1.500000,5.500000,1,5,2,children_playing
8,100263-2-0-36.wav,100263,18.000000,22.000000,1,5,2,children_playing
9,100648-1-0-0.wav,100648,4.823402,5.471927,2,10,1,car_horn


In [3]:
print("The original data containes {0} audio files.".format(data.shape[0]))

# Filter data less than 3 seconds 
dataorg = data[['slice_file_name', 'fold' ,'classID', 'class']]
data3s = data[['slice_file_name', 'fold' ,'classID', 'class']][ data['end']-data['start'] >= 3 ]

print("The data we will work with containes {0} audio files of 3 seconds length.".format(data3s.shape[0]))

data3s.head(10)

The original data containes 8732 audio files.
The data we will work with containes 7468 audio files of 3 seconds length.


,slice_file_name,fold,classID,class
1,100263-2-0-117.wav,5,2,children_playing
2,100263-2-0-121.wav,5,2,children_playing
3,100263-2-0-126.wav,5,2,children_playing
4,100263-2-0-137.wav,5,2,children_playing
5,100263-2-0-143.wav,5,2,children_playing
6,100263-2-0-161.wav,5,2,children_playing
7,100263-2-0-3.wav,5,2,children_playing
8,100263-2-0-36.wav,5,2,children_playing
14,100652-3-0-0.wav,2,3,dog_bark
15,100652-3-0-1.wav,2,3,dog_bark


Different Classes are:
0. air_conditioner  
1. car_horn  
2. children_playing  
3. dog_bark  
4. drilling  
5. engine_idling
6. gun_shot
7. jackhammer
8. siren
9. street_music

Iterate over all samples in _valid_.
For every sample, construct the (128,128) spectrogram

In [5]:
data3s['path'] = 'fold' + data3s['fold'].astype('str') + '/' + data3s['slice_file_name'].astype('str')

In [ ]:
dataset = []

for row in data3s.itertuples():
    y, sr = librosa.load('data/UrbanSound8K/' + row.path, duration=2.97)  
    ps = librosa.feature.melspectrogram(y=y, sr=sr)
    if ps.shape != (128, 128): continue
    dataset.append( (ps, row.classID) )

In [10]:
random.shuffle(dataset)
# X_train, X_test, y_train, y_test = train_test_split(dataset, test_size = .2)

train = dataset[:7000]
test = dataset[7000:]

X_train, y_train = zip(*train)
X_test, y_test = zip(*test)

# Reshape for CNN input
X_train = np.array([x.reshape( (128, 128, 1) ) for x in X_train])
X_test = np.array([x.reshape( (128, 128, 1) ) for x in X_test])

# One-Hot encoding for classes
y_train = np.array(keras.utils.to_categorical(y_train, 10))
y_test = np.array(keras.utils.to_categorical(y_test, 10))

In [11]:
model = keras.models.Sequential()
input_shape=(128, 128, 1)

model.add(keras.layers.Conv2D(24, (5, 5), strides=(1, 1), input_shape=input_shape))
model.add(keras.layers.MaxPooling2D((4, 2), strides=(4, 2)))
model.add(keras.layers.Activation('relu'))

model.add(keras.layers.Conv2D(48, (5, 5), padding="valid"))
model.add(keras.layers.MaxPooling2D((4, 2), strides=(4, 2)))
model.add(keras.layers.Activation('relu'))

model.add(keras.layers.Conv2D(48, (5, 5), padding="valid"))
model.add(keras.layers.Activation('relu'))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(rate=0.5))

model.add(keras.layers.Dense(64))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dropout(rate=0.5))

model.add(keras.layers.Dense(10))
model.add(keras.layers.Activation('softmax'))

In [12]:
model.compile(
	optimizer="Adam",
	loss="categorical_crossentropy",
	metrics=['accuracy'])

model.fit(
	x=X_train, 
	y=y_train,
    epochs=12,
    batch_size=500,
    validation_data= (X_test, y_test))

score = model.evaluate(x=X_test, y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 7000 samples, validate on 467 samples
Epoch 1/12
7000/7000 [==============================] - 63s 9ms/step - loss: 3.0229 - acc: 0.1746 - val_loss: 2.1602 - val_acc: 0.2420
Epoch 2/12
7000/7000 [==============================] - 63s 9ms/step - loss: 2.2378 - acc: 0.2257 - val_loss: 2.0827 - val_acc: 0.3212
Epoch 3/12
7000/7000 [==============================] - 62s 9ms/step - loss: 2.1222 - acc: 0.2904 - val_loss: 2.0035 - val_acc: 0.3662
Epoch 4/12
7000/7000 [==============================] - 62s 9ms/step - loss: 2.0318 - acc: 0.3277 - val_loss: 1.8970 - val_acc: 0.4176
Epoch 5/12
7000/7000 [==============================] - 64s 9ms/step - loss: 1.9078 - acc: 0.3296 - val_loss: 1.7871 - val_acc: 0.3812
Epoch 6/12
7000/7000 [==============================] - 63s 9ms/step - loss: 1.8267 - acc: 0.3560 - val_loss: 1.7805 - val_acc: 0.4325
Epoch 7/12
7000/7000 [==============================] - 62s 9ms/step - loss: 1.7324 - acc: 0.3944 - val_loss: 1.6715 - val_acc: 0.4604
Epoch 8/

In [31]:
# predict

# what to predict
k = np.array(X_train[4545]) #seven

# shape
y = k.reshape(1, 128, 128, 1)

# predict
prediction = model.predict(y)
print(prediction)

# predict class
model.predict_classes(y)

[[1.3292677e-07 1.4357405e-06 4.9620826e-04 4.1916853e-04 2.9710193e-06
  5.0701931e-07 1.3825893e-10 6.6827660e-08 9.9461669e-01 4.4628559e-03]]


array([8])

In [42]:
y, sr = librosa.load('data/myAudioFiles/4.drilling.m4a', duration=2.97)  
ps = librosa.feature.melspectrogram(y=y, sr=sr)

In [43]:
k = np.array(ps) #seven

# shape
y = k.reshape(1, 128, 128, 1)

# predict
prediction = model.predict(y)
print(prediction)

# predict class
model.predict_classes(y)

[[0.09900478 0.01838225 0.33766407 0.11481174 0.04258447 0.06590442
  0.0031979  0.01311154 0.18226707 0.12307175]]


array([2])